In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

nice_colors = {
    "TES": "orange",
    "Battery": "darkgrey",
    "H2 storage": "green",
    "Electrolysis": "lightcoral",
    "HVDC": "blue",
    "H2 Pipeline": "blue",
}

# magic number from efficiencies.csv; report TES storage and H2 storage in MWh electricity rather than heat
efficiency_csp_power_block = 0.41
efficiency_h2_ccgt = 0.58

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
]
df = df.loc[df["category"].isin(["capacity factor", "installed capacity"])]

# Extract technology to aggreagate/group by later
df["technology"] = df["subcategory"].str.extract("^(.*)\s\d*|\(.*\)$")

# Calculate weighted CF for technology (rather than individual PyPSA components)
# Transformations and intermediary data storage
df = df.set_index(["scenario", "subcategory"])
data = df.query("category == 'installed capacity'")
data = data[["technology", "value"]]
data = data.rename(columns={"value": "capacity"})
data["capacity factor"] = df.query("category == 'capacity factor'")["value"]
data["CF*CAP"] = data["capacity factor"] * data["capacity"]

# this data will be used for plotting
plot_df = (
    data.groupby(["scenario", "technology"])["CF*CAP"].sum()
    / data.groupby(["scenario", "technology"])["capacity"].sum()
)
plot_df = plot_df.to_frame("weighted CF")
plot_df["total capacity"] = data.groupby(["scenario", "technology"])["capacity"].sum()
plot_df["total capacity"] /= 1e3  # in GWh
plot_df = plot_df.reset_index()

# CSP-TES and H2 storage in MWh elec instead of heat
plot_df.loc[
    plot_df["technology"] == "csp-tower TES", "total capacity"
] *= efficiency_csp_power_block
if "h2" in snakemake.wildcards["esc"]:
    plot_df.loc[
        plot_df["technology"] == "hydrogen storage tank type 1", "total capacity"
    ] *= efficiency_h2_ccgt

plot_df["flexibility"] = plot_df["scenario"].str.replace("_.*", "", regex=True)
plot_df["csp"] = plot_df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quaterly",
    "annually": "Yearly",
}

plot_df["flexibility"] = plot_df.flexibility.replace(flexibilities)

plot_df["flexibility"] = pd.Categorical(
    plot_df.flexibility, categories=flexibilities.values(), ordered=True
)

plot_df = plot_df.sort_values("flexibility")
plot_df = plot_df.drop(columns="scenario")

# Cleaning: Don't report CF and capacity for negligible capacities close to zero
plot_df.loc[plot_df["total capacity"] < 1, "weighted CF"] = 0
plot_df.loc[plot_df["total capacity"] < 1, "total capacity"] = 0

plot_df = plot_df.set_index(["csp", "flexibility"])

# Self-evident; also unnecessary comment.
technologies_to_plot = [
    "Battery",
    "TES",
    "HVDC",
    "H2 storage",
    "H2 Pipeline",
    "Electrolysis",
]

# Nicer names for technologies
plot_df.loc[plot_df["technology"] == "HVDC", "technology"] = "HVDC"
plot_df.loc[plot_df["technology"] == "battery storage", "technology"] = "Battery"
plot_df.loc[plot_df["technology"] == "csp-tower TES", "technology"] = "TES"
plot_df.loc[
    plot_df["technology"] == "hydrogen storage tank type 1", "technology"
] = "H2 storage"
plot_df.loc[
    plot_df["technology"] == "H2 (g) fill compressor station", "technology"
] = "H2 Pipeline"
plot_df.loc[plot_df["technology"] == "electrolysis", "technology"] = "Electrolysis"

## Plot
fig, axes = plt.subplots(2, 1, figsize=(6, 5), sharex=True)

# upper plot w/o CSP
axl = axes[0]
axr = axl.twinx()

csp = False
df = plot_df.query(
    "csp == @csp and technology.isin(@technologies_to_plot)", engine="python"
).reset_index()
plot_data = df.pivot(
    index="flexibility", columns="technology", values="total capacity"
)[[t for t in technologies_to_plot if t in df["technology"].unique()]]
plot_data.plot.bar(ax=axl, color=[nice_colors[k] for k in plot_data.columns])
plot_data = df.pivot(index="flexibility", columns="technology", values="weighted CF")[
    [t for t in technologies_to_plot if t in df["technology"].unique()]
]
plot_data.plot.line(
    ax=axr,
    marker="o",
    markerfacecolor="white",
    color=[nice_colors[k] for k in plot_data.columns],
)

axr.set_ylim(0, 1.05)
axr.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
axr.set_ylabel("Utilisation factor [p.u.]")

axl.set_ylim(0, 160 * 1.05)  # Magic values for consistency for all ESCs
axl.set_yticks(np.linspace(0, 160, 5))
axl.set_ylabel("Capacity [$\mathregular{GWh}_{e}$, GW]")

axl.set_xticklabels(axl.get_xticklabels(), rotation=0, fontsize="small")
axl.tick_params(axis="both", which="major", labelsize="small")
axr.tick_params(axis="both", which="major", labelsize="small")

# Remove legend
axl.legend().remove()
axr.legend().remove()

# upper plot w/o CSP
axl = axes[1]
axr = axl.twinx()

csp = True
df = plot_df.query(
    "csp == @csp and technology.isin(@technologies_to_plot)", engine="python"
).reset_index()
plot_data = df.pivot(
    index="flexibility", columns="technology", values="total capacity"
)[[t for t in technologies_to_plot if t in df["technology"].unique()]]
plot_data.plot.bar(
    ax=axl, hatch="///", color=[nice_colors[k] for k in plot_data.columns]
)
plot_data = df.pivot(index="flexibility", columns="technology", values="weighted CF")[
    [t for t in technologies_to_plot if t in df["technology"].unique()]
]
plot_data.plot.line(
    ax=axr,
    marker="o",
    markerfacecolor="white",
    color=[nice_colors[k] for k in plot_data.columns],
)


axr.set_ylim(0, 1.05)
axr.set_yticks([0, 0.25, 0.5, 0.75, 1.0])
axr.set_ylabel("Utilisation factor [p.u.]")

axl.set_xlim(
    -0.5,
)
axl.set_xlabel("Flexibility to shift load")
axl.set_ylim(0, 160 * 1.05)  # Magic values for consistency for all ESCs
axl.set_yticks(np.linspace(0, 160, 5))
axl.set_ylabel("Capacity [$\mathregular{GWh}_{e}$, GW]")

axl.set_xticklabels(axl.get_xticklabels(), rotation=0, fontsize="small")
axl.tick_params(axis="both", which="major", labelsize="small")
axr.tick_params(axis="both", which="major", labelsize="small")

# Add legend handles/labels to upper plot (1 technology less b/c of missing TES/CSP)
# use and combine handles, matplotlib overlaps them automatically
axes[0].legend(
    handles=list(
        zip(axl.get_legend_handles_labels()[0], axr.get_legend_handles_labels()[0])
    ),
    labels=axes[1].get_legend_handles_labels()[1],
    fontsize="small",
    ncols=len(technologies_to_plot),
    columnspacing=1.0,
    bbox_to_anchor=(0, 1),
    loc="lower left",
    # ncols=np.ceil(len(axes[1].get_legend_handles_labels()[1])/2),
)
# ... then remove legend from lower plot
axl.legend().remove()
axr.legend().remove()

fig.tight_layout(pad=0)

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")